## Image Alteration Detection using Pretrained CNN (MobileNetV2)

#### Import libraries

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

#### Defining required parameters

In [2]:
# Parameters
IMG_SIZE = (128, 128)  # Smaller input for faster training
BATCH_SIZE = 32
EPOCHS = 10

# Paths
train_dir = 'dataset/train'  
val_split = 0.2

#### Preparing data

In [3]:
train_gen = ImageDataGenerator(
    rescale=1./255,
    validation_split=val_split,
    horizontal_flip=True,
    rotation_range=15,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1
)

train_data = train_gen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

val_data = train_gen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

Found 80000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.


#### Building Model

In [4]:
base_model = MobileNetV2(input_shape=IMG_SIZE + (3,), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze base

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Exception: URL fetch failure on https://storage.googleapis.com/tensorflow/keras-applications/mobilenet_v2/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_128_no_top.h5: 403 -- Forbidden

#### Training model

In [13]:
early_stop = EarlyStopping(patience=3, restore_best_weights=True)

history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS,
    callbacks=[early_stop]
)

Epoch 1/10
 409/5000 ━━━━━━━━━━━━━━━━━━━━ 21:36 282ms/step - accuracy: 0.5529 - loss: 0.6884

KeyboardInterrupt: 

#### Evaluating

In [ ]:
val_data.reset()
preds = model.predict(val_data)
pred_classes = (preds > 0.5).astype("int32").flatten()
true_classes = val_data.classes

print("Classification Report:\n", classification_report(true_classes, pred_classes, target_names=["Fake", "Real"]))
print("Confusion Matrix:\n", confusion_matrix(true_classes, pred_classes))

#### Plot Training Curves

In [ ]:
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.legend()
plt.title("Accuracy")

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title("Loss")

plt.tight_layout()
plt.show()